# **IMPORTS**


In [52]:
import pandas as pd 
import re
from collections import Counter
import os

# **LA LIGA**


### **Import**


In [53]:
file = pd.read_csv("Datasets/el_clasico_matches_la_liga.csv")
df = pd.DataFrame(file)

In [54]:
df.tail()

,No.,Date,Matchweek,Home team,Away team,Score (FT/HT),Goals (home),Goals (away)
185,186,19 March 2023,26,Barcelona,Real Madrid,2–1 (1–1),"Roberto (45), Kessié (90+1)",Araújo (9 o.g.)
186,187,28 October 2023,11,Barcelona,Real Madrid,1–2 (1–0),Gündoğan (6),"Bellingham (68, 90+2)"
187,188,21 April 2024,32,Real Madrid,Barcelona,3–2 (1–1),"Vinícius (18 p.), Vázquez (73), Bellingham (90+1)","Christensen (6), López (69)"
188,189,26 October 2024,11,Real Madrid,Barcelona,0–4 (0–0),NaN,"Lewandowski (54, 56), Yamal (77), Raphinha (84)"
189,190,11 May 2025,35,Barcelona,Real Madrid,NaN,NaN,NaN


### **Change to Date Format**


In [55]:
df['Date'] = pd.to_datetime(df['Date'], format='%d %B %Y')
df['Year'] = df['Date'].dt.year

In [56]:
df.head()

,No.,Date,Matchweek,Home team,Away team,Score (FT/HT),Goals (home),Goals (away),Year
0,1,1929-02-17,2,Barcelona,Real Madrid,1–2 (0–1),Parera (70),"Morera (10, 55)",1929
1,2,1929-05-09,11,Real Madrid,Barcelona,0–1 (0–0),NaN,Sastre (83),1929
2,3,1930-01-26,9,Barcelona,Real Madrid,1–4 (0–3),Bestit (63),"Rubio (10, 37), F. López (17), Lazcano (71)",1930
3,4,1930-03-30,18,Real Madrid,Barcelona,5–1 (3–0),"Rubio (5, 23), Lazcano (42, 68, 72)",Goiburu (84),1930
4,5,1931-02-01,9,Real Madrid,Barcelona,0–0,NaN,NaN,1931


In [57]:
df = df.dropna(subset=["Score (FT/HT)"])

In [58]:
df.shape

(189, 9)

In [59]:
df.tail()

,No.,Date,Matchweek,Home team,Away team,Score (FT/HT),Goals (home),Goals (away),Year
184,185,2022-10-16,9,Real Madrid,Barcelona,3–1 (2–0),"Benzema (12), Valverde (35), Rodrygo (90+1 p.)",Torres (83),2022
185,186,2023-03-19,26,Barcelona,Real Madrid,2–1 (1–1),"Roberto (45), Kessié (90+1)",Araújo (9 o.g.),2023
186,187,2023-10-28,11,Barcelona,Real Madrid,1–2 (1–0),Gündoğan (6),"Bellingham (68, 90+2)",2023
187,188,2024-04-21,32,Real Madrid,Barcelona,3–2 (1–1),"Vinícius (18 p.), Vázquez (73), Bellingham (90+1)","Christensen (6), López (69)",2024
188,189,2024-10-26,11,Real Madrid,Barcelona,0–4 (0–0),NaN,"Lewandowski (54, 56), Yamal (77), Raphinha (84)",2024


### **Goals**


In [60]:
df["Home Goals"] = df["Score (FT/HT)"].str.extract(r"(\d+)–(\d+)")[0].astype(int)
df["Away Goals"] = df["Score (FT/HT)"].str.extract(r"(\d+)–(\d+)")[1].astype(int)

In [61]:
df["Barcelona Goals"] = df.apply(lambda row: row["Home Goals"] if row["Home team"] == "Barcelona" else row["Away Goals"], axis=1)
df["Real Madrid Goals"] = df.apply(lambda row: row["Away Goals"] if row["Home team"] == "Barcelona" else row["Home Goals"], axis=1)

### **Winner Column**


In [62]:
def determine_winner(row):
    if row["Barcelona Goals"] > row["Real Madrid Goals"]:
        return "Barcelona"
    elif row["Barcelona Goals"] < row["Real Madrid Goals"]:
        return "Real Madrid"
    else:
        return "Draw"



In [63]:
df["Winner"] = df.apply(determine_winner, axis=1)

### **Venue Mapping**


In [64]:
venue_map = {
    "Barcelona": "Camp Nou",
    "Real Madrid": "Santiago Bernabéu Stadium"
}


In [65]:
df["Venue"] = df["Home team"].map(venue_map)

In [66]:
df.head()

,No.,Date,Matchweek,Home team,Away team,Score (FT/HT),Goals (home),Goals (away),Year,Home Goals,Away Goals,Barcelona Goals,Real Madrid Goals,Winner,Venue
0,1,1929-02-17,2,Barcelona,Real Madrid,1–2 (0–1),Parera (70),"Morera (10, 55)",1929,1,2,1,2,Real Madrid,Camp Nou
1,2,1929-05-09,11,Real Madrid,Barcelona,0–1 (0–0),NaN,Sastre (83),1929,0,1,1,0,Barcelona,Santiago Bernabéu Stadium
2,3,1930-01-26,9,Barcelona,Real Madrid,1–4 (0–3),Bestit (63),"Rubio (10, 37), F. López (17), Lazcano (71)",1930,1,4,1,4,Real Madrid,Camp Nou
3,4,1930-03-30,18,Real Madrid,Barcelona,5–1 (3–0),"Rubio (5, 23), Lazcano (42, 68, 72)",Goiburu (84),1930,5,1,1,5,Real Madrid,Santiago Bernabéu Stadium
4,5,1931-02-01,9,Real Madrid,Barcelona,0–0,NaN,NaN,1931,0,0,0,0,Draw,Santiago Bernabéu Stadium


### **Scorers**


In [67]:
def scorers(df):
    df["Barcelona Scorers"] = df.apply(lambda row: row["Goals (home)"] if row["Home team"] == "Barcelona" else row["Goals (away)"], axis=1)
    df["Real Madrid Scorers"] = df.apply(lambda row: row["Goals (away)"] if row["Home team"] == "Barcelona" else row["Goals (home)"], axis=1)
    return df

In [68]:
df = scorers(df)

In [69]:
df.head()

,No.,Date,Matchweek,Home team,Away team,Score (FT/HT),Goals (home),Goals (away),Year,Home Goals,Away Goals,Barcelona Goals,Real Madrid Goals,Winner,Venue,Barcelona Scorers,Real Madrid Scorers
0,1,1929-02-17,2,Barcelona,Real Madrid,1–2 (0–1),Parera (70),"Morera (10, 55)",1929,1,2,1,2,Real Madrid,Camp Nou,Parera (70),"Morera (10, 55)"
1,2,1929-05-09,11,Real Madrid,Barcelona,0–1 (0–0),NaN,Sastre (83),1929,0,1,1,0,Barcelona,Santiago Bernabéu Stadium,Sastre (83),NaN
2,3,1930-01-26,9,Barcelona,Real Madrid,1–4 (0–3),Bestit (63),"Rubio (10, 37), F. López (17), Lazcano (71)",1930,1,4,1,4,Real Madrid,Camp Nou,Bestit (63),"Rubio (10, 37), F. López (17), Lazcano (71)"
3,4,1930-03-30,18,Real Madrid,Barcelona,5–1 (3–0),"Rubio (5, 23), Lazcano (42, 68, 72)",Goiburu (84),1930,5,1,1,5,Real Madrid,Santiago Bernabéu Stadium,Goiburu (84),"Rubio (5, 23), Lazcano (42, 68, 72)"
4,5,1931-02-01,9,Real Madrid,Barcelona,0–0,NaN,NaN,1931,0,0,0,0,Draw,Santiago Bernabéu Stadium,NaN,NaN


In [70]:
def extract_own_goals(row, team_col, scorers_col):
        own_goals = []
        if pd.notna(row[scorers_col]):
            scorers = str(row[scorers_col]).split(', ')
            own_goals = [scorer.split(' ')[0] for scorer in scorers if 'o.g.' in scorer]
        return ', '.join(own_goals) if own_goals else None

def split_multiple_goals(text):
    """
    Splits multiple goals by same player into separate entries.
    Example: "Lewandowski (54, 56)" -> ["Lewandowski (54)", "Lewandowski (56)"]
    """
    if pd.isna(text):
        return None
    
    results = []
    # Find patterns like "Player (min1, min2)" or "Player (min1, min2+extra)"
    matches = re.finditer(r'(\w+)\s*\(((?:\d+(?:\+\d+)?(?:\s*p\.)?(?:,\s*)?)+)\)', text)
    
    for match in matches:
        player = match.group(1)
        minutes = match.group(2).split(',')
        for minute in minutes:
            minute = minute.strip()
            if minute:  # Skip empty strings
                results.append(f"{player} ({minute})")
    
    return results

def remove_minutes(text):
    """
    Removes minutes from player names.
    Example: "Christensen (6)" -> "Christensen"
    """
    if pd.isna(text):
        return None
    
    # Replace patterns like "(number)" or "(number+number)"
    return re.sub(r'\s*\([^)]*\)', '', text)

# Example usage with a pandas DataFrame:
def transform_match_data(df):
    """
    Applies all transformations to the DataFrame
    """
    # Create new columns for own goals
    df['Real_Madrid_Own_Goals'] = df.apply(lambda x: extract_own_goals(x, 'Home team', 'Barcelona Scorers'), axis=1) 
    df['Barcelona_Own_Goals'] = df.apply(lambda x: extract_own_goals(x, 'Away team', 'Real Madrid Scorers'), axis=1)
    
    # Split multiple goals
    df['Barcelona Scorers'] = df['Barcelona Scorers'].apply(lambda x: ', '.join(split_multiple_goals(x)) if not pd.isna(x) else None)
    df['Real Madrid Scorers'] = df['Real Madrid Scorers'].apply(lambda x: ', '.join(split_multiple_goals(x)) if not pd.isna(x) else None)
    
    # Remove minutes
    df['Barcelona Scorers Clean'] = df['Barcelona Scorers'].apply(remove_minutes)
    df['Real Madrid Scorers Clean'] = df['Real Madrid Scorers'].apply(remove_minutes)
    
    return df

In [71]:
df = transform_match_data(df)

In [72]:
df

,No.,Date,Matchweek,Home team,Away team,Score (FT/HT),Goals (home),Goals (away),Year,Home Goals,...,Barcelona Goals,Real Madrid Goals,Winner,Venue,Barcelona Scorers,Real Madrid Scorers,Real_Madrid_Own_Goals,Barcelona_Own_Goals,Barcelona Scorers Clean,Real Madrid Scorers Clean
0,1,1929-02-17,2,Barcelona,Real Madrid,1–2 (0–1),Parera (70),"Morera (10, 55)",1929,1,...,1,2,Real Madrid,Camp Nou,Parera (70),"Morera (10), Morera (55)",None,None,Parera,"Morera, Morera"
1,2,1929-05-09,11,Real Madrid,Barcelona,0–1 (0–0),NaN,Sastre (83),1929,0,...,1,0,Barcelona,Santiago Bernabéu Stadium,Sastre (83),None,None,None,Sastre,None
2,3,1930-01-26,9,Barcelona,Real Madrid,1–4 (0–3),Bestit (63),"Rubio (10, 37), F. López (17), Lazcano (71)",1930,1,...,1,4,Real Madrid,Camp Nou,Bestit (63),"Rubio (10), Rubio (37), López (17), Lazcano (71)",None,None,Bestit,"Rubio, Rubio, López, Lazcano"
3,4,1930-03-30,18,Real Madrid,Barcelona,5–1 (3–0),"Rubio (5, 23), Lazcano (42, 68, 72)",Goiburu (84),1930,5,...,1,5,Real Madrid,Santiago Bernabéu Stadium,Goiburu (84),"Rubio (5), Rubio (23), Lazcano (42), Lazcano (...",None,None,Goiburu,"Rubio, Rubio, Lazcano, Lazcano, Lazcano"
4,5,1931-02-01,9,Real Madrid,Barcelona,0–0,NaN,NaN,1931,0,...,0,0,Draw,Santiago Bernabéu Stadium,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,185,2022-10-16,9,Real Madrid,Barcelona,3–1 (2–0),"Benzema (12), Valverde (35), Rodrygo (90+1 p.)",Torres (83),2022,3,...,1,3,Real Madrid,Santiago Bernabéu Stadium,Torres (83),"Benzema (12), Valverde (35), Rodrygo (90+1 p.)",None,None,Torres,"Benzema, Valverde, Rodrygo"
185,186,2023-03-19,26,Barcelona,Real Madrid,2–1 (1–1),"Roberto (45), Kessié (90+1)",Araújo (9 o.g.),2023,2,...,2,1,Barcelona,Camp Nou,"Roberto (45), Kessié (90+1)",,None,Araújo,"Roberto, Kessié",
186,187,2023-10-28,11,Barcelona,Real Madrid,1–2 (1–0),Gündoğan (6),"Bellingham (68, 90+2)",2023,1,...,1,2,Real Madrid,Camp Nou,Gündoğan (6),"Bellingham (68), Bellingham (90+2)",None,None,Gündoğan,"Bellingham, Bellingham"
187,188,2024-04-21,32,Real Madrid,Barcelona,3–2 (1–1),"Vinícius (18 p.), Vázquez (73), Bellingham (90+1)","Christensen (6), López (69)",2024,3,...,2,3,Real Madrid,Santiago Bernabéu Stadium,"Christensen (6), López (69)","Vinícius (18 p.), Vázquez (73), Bellingham (90+1)",None,None,"Christensen, López","Vinícius, Vázquez, Bellingham"


### **New Dataframe**


In [73]:
columns={
            "Barcelona Goals": "Barcelona_Goals", 
            "Real Madrid Goals": "Real_Madrid_Goals",
            "Barcelona Scorers Clean" : "Barcelona_Scorers",
            "Real Madrid Scorers Clean": "Real_Madrid_Scorers"
         }
new_df = df.filter(items=["Date","Year", "Venue", "Barcelona Goals", "Real Madrid Goals", "Winner", "Real_Madrid_Own_Goals", "Barcelona_Own_Goals","Barcelona Scorers Clean", "Real Madrid Scorers Clean"]).rename(columns = columns)

In [74]:
new_df.head()

,Date,Year,Venue,Barcelona_Goals,Real_Madrid_Goals,Winner,Real_Madrid_Own_Goals,Barcelona_Own_Goals,Barcelona_Scorers,Real_Madrid_Scorers
0,1929-02-17,1929,Camp Nou,1,2,Real Madrid,None,None,Parera,"Morera, Morera"
1,1929-05-09,1929,Santiago Bernabéu Stadium,1,0,Barcelona,None,None,Sastre,None
2,1930-01-26,1930,Camp Nou,1,4,Real Madrid,None,None,Bestit,"Rubio, Rubio, López, Lazcano"
3,1930-03-30,1930,Santiago Bernabéu Stadium,1,5,Real Madrid,None,None,Goiburu,"Rubio, Rubio, Lazcano, Lazcano, Lazcano"
4,1931-02-01,1931,Santiago Bernabéu Stadium,0,0,Draw,None,None,None,None


In [75]:
new_df['League'] = 'La Liga'
new_df.to_csv("Datasets/Cleaned_ds/la_liga.csv", index=False)

# **UEFA**


#### **Import**


In [71]:
file = pd.read_csv("Datasets/el_clasico_matches_uefa.csv")
df = pd.DataFrame(file)

In [72]:
df.head()

,Season,Round,Round.1,Home team,Away team,Score (FT/HT),Goals (home),Goals (away)
0,1959–60,Semi-finals,First leg,Real Madrid,Barcelona,3–1 (2–1),"Di Stéfano (17, 84), Puskás (28)",Martínez (37)
1,1959–60,Semi-finals,Second leg,Barcelona,Real Madrid,1–3 (1–1),Kocsis (89),"Puskás (25, 75), Gento (68)"
2,1960–61,First round,First leg,Real Madrid,Barcelona,2–2 (2–1),"Mateos (3), Gento (33)","Luis Suárez (27, 87 p.)"
3,1960–61,First round,Second leg,Barcelona,Real Madrid,2–1 (1–0),"Vergés (33), Evaristo (82)",Canário (87)
4,2001–02,Semi-finals,First leg,Barcelona,Real Madrid,0–2 (0–0),NaN,"Zidane (55), McManaman (90+2)"


#### **Scorers**


In [29]:
df = scorers(df)
df = transform_match_data(df)

In [30]:
df.head()

,Season,Round,Round.1,Home team,Away team,Score (FT/HT),Goals (home),Goals (away),Barcelona Scorers,Real Madrid Scorers,Real_Madrid_Own_Goals,Barcelona_Own_Goals,Barcelona Scorers Clean,Real Madrid Scorers Clean
0,1959–60,Semi-finals,First leg,Real Madrid,Barcelona,3–1 (2–1),"Di Stéfano (17, 84), Puskás (28)",Martínez (37),Martínez (37),"Stéfano (17), Stéfano (84), Puskás (28)",None,None,Martínez,"Stéfano, Stéfano, Puskás"
1,1959–60,Semi-finals,Second leg,Barcelona,Real Madrid,1–3 (1–1),Kocsis (89),"Puskás (25, 75), Gento (68)",Kocsis (89),"Puskás (25), Puskás (75), Gento (68)",None,None,Kocsis,"Puskás, Puskás, Gento"
2,1960–61,First round,First leg,Real Madrid,Barcelona,2–2 (2–1),"Mateos (3), Gento (33)","Luis Suárez (27, 87 p.)",,"Mateos (3), Gento (33)",None,None,,"Mateos, Gento"
3,1960–61,First round,Second leg,Barcelona,Real Madrid,2–1 (1–0),"Vergés (33), Evaristo (82)",Canário (87),"Vergés (33), Evaristo (82)",Canário (87),None,None,"Vergés, Evaristo",Canário
4,2001–02,Semi-finals,First leg,Barcelona,Real Madrid,0–2 (0–0),NaN,"Zidane (55), McManaman (90+2)",None,"Zidane (55), McManaman (90+2)",None,None,None,"Zidane, McManaman"


#### **Goals**


In [31]:
print(df.columns.tolist())

['Season', 'Round', 'Round.1', 'Home team', 'Away team', 'Score\xa0(FT/HT)', 'Goals (home)', 'Goals (away)', 'Barcelona Scorers', 'Real Madrid Scorers', 'Real_Madrid_Own_Goals', 'Barcelona_Own_Goals', 'Barcelona Scorers Clean', 'Real Madrid Scorers Clean']


In [32]:
def goals(df):
    df["Home Goals"] = df["Score\xa0(FT/HT)"].str.extract(r"(\d+)–(\d+)")[0].astype(int)
    df["Away Goals"] = df["Score\xa0(FT/HT)"].str.extract(r"(\d+)–(\d+)")[1].astype(int)
    df["Barcelona Goals"] = df.apply(lambda row: row["Home Goals"] if row["Home team"] == "Barcelona" else row["Away Goals"], axis=1)
    df["Real Madrid Goals"] = df.apply(lambda row: row["Away Goals"] if row["Home team"] == "Barcelona" else row["Home Goals"], axis=1)
    return df

In [33]:
df = goals(df)

#### **Winner**


In [34]:
df["Winner"] = df.apply(determine_winner, axis=1)

#### **Venue Mapping**


In [35]:
df["Venue"] = df["Home team"].map(venue_map)

#### **New DF**


In [36]:
columns={
            "Season": "Year",
            "Barcelona Goals": "Barcelona_Goals", 
            "Real Madrid Goals": "Real_Madrid_Goals",
            "Barcelona Scorers Clean" : "Barcelona_Scorers",
            "Real Madrid Scorers Clean": "Real_Madrid_Scorers"
         }
new_df = df.filter(items=["Season", "Venue", "Barcelona Goals", "Real Madrid Goals", "Winner", "Real_Madrid_Own_Goals", "Barcelona_Own_Goals","Barcelona Scorers Clean", "Real Madrid Scorers Clean"]).rename(columns = columns)

In [37]:
new_df.head()

,Year,Venue,Barcelona_Goals,Real_Madrid_Goals,Winner,Real_Madrid_Own_Goals,Barcelona_Own_Goals,Barcelona_Scorers,Real_Madrid_Scorers
0,1959–60,Santiago Bernabéu Stadium,1,3,Real Madrid,None,None,Martínez,"Stéfano, Stéfano, Puskás"
1,1959–60,Camp Nou,1,3,Real Madrid,None,None,Kocsis,"Puskás, Puskás, Gento"
2,1960–61,Santiago Bernabéu Stadium,2,2,Draw,None,None,,"Mateos, Gento"
3,1960–61,Camp Nou,2,1,Barcelona,None,None,"Vergés, Evaristo",Canário
4,2001–02,Camp Nou,0,2,Real Madrid,None,None,None,"Zidane, McManaman"


In [38]:
new_df['League'] = 'UEFA Championship'
new_df.to_csv("Datasets/Cleaned_ds/uefa.csv", index=False)

# **COPA DEL REY**


In [308]:
file = pd.read_csv("Datasets/el_clasico_matches_copa_del_rey.csv")
df = pd.DataFrame(file)

In [309]:
df.tail()

,Season,Round,Round.1,Home team,Away team,Score (FT/HT),Goals (home),Goals (away)
32,2013–14,Final,Final,Real Madrid*,Barcelona,2–1 (1–0),"Di María (11), Bale (85)",Bartra (69)
33,2018–19,Semi-finals,First leg,Barcelona,Real Madrid,1–1 (0–1),Malcom (57),Vázquez (6)
34,2018–19,Semi-finals,Second leg,Real Madrid,Barcelona,0–3 (0–0),NaN,"L. Suárez (50, 73 p.), Varane (69 o.g.)"
35,2022–23,Semi-finals,First leg,Real Madrid,Barcelona,0–1 (0–1),NaN,Militão (26 o.g.)
36,2022–23,Semi-finals,Second leg,Barcelona,Real Madrid,0–4 (0–1),NaN,"Vinícius (45+1), Benzema (50, 58 p., 80)"


In [311]:
df['Season'].iloc[10:20]

10    1954
11    1957
12    1957
13    1959
14    1959
15    1962
16    1962
17    1968
18    1970
19    1970
Name: Season, dtype: int64

In [310]:
def clean_year(year):
    if isinstance(year, str):
        # Replace any non-standard dashes with a normal hyphen
        year = year.replace('â€“', '-').replace('–', '-').replace('—', '-')
        
        # If there's a hyphen, take the second part and format it correctly
        if '-' in year:
            year = year.split('-')[1]  # Take the second part (e.g., '59' from '1958-59')
            if len(year) == 2:  # If it's a two-digit year like '59'
                return '19' + year  # Convert '59' to '1959'
    
    return year  # Return the original value if no hyphen is found

# Apply the function
df['Season'] = df['Season'].apply(clean_year)

# Convert to integer type if needed
df['Season'] = df['Season'].astype(int)

In [312]:
df = scorers(df)
df = transform_match_data(df)

In [313]:
df = goals(df)

In [314]:
df["Winner"] = df.apply(determine_winner, axis=1)

In [315]:
df["Venue"] = df["Home team"].map(venue_map)

In [316]:
columns={
            "Season": "Year",
            "Barcelona Goals": "Barcelona_Goals", 
            "Real Madrid Goals": "Real_Madrid_Goals",
            "Barcelona Scorers Clean" : "Barcelona_Scorers",
            "Real Madrid Scorers Clean": "Real_Madrid_Scorers"
         }
new_df = df.filter(items=["Season", "Venue", "Barcelona Goals", "Real Madrid Goals", "Winner", "Real_Madrid_Own_Goals", "Barcelona_Own_Goals","Barcelona Scorers Clean", "Real Madrid Scorers Clean"]).rename(columns = columns)

In [317]:
new_df.head()

,Year,Venue,Barcelona_Goals,Real_Madrid_Goals,Winner,Real_Madrid_Own_Goals,Barcelona_Own_Goals,Barcelona_Scorers,Real_Madrid_Scorers
0,1916,Camp Nou,2,1,Barcelona,None,None,"Alcántara, Martínez",Petit
1,1916,Santiago Bernabéu Stadium,1,4,Real Madrid,None,None,Martínez,Petit
2,1916,Santiago Bernabéu Stadium,6,6,Draw,None,None,"Alcántara, Alcántara, Alcántara, Mallorquí, Ba...","Belaunde, Belaunde, Belaunde"
3,1916,Santiago Bernabéu Stadium,2,4,Real Madrid,None,None,"Martínez, Martínez","Bernabéu, Zabalo, Aranguren, Aranguren"
4,1926,Santiago Bernabéu Stadium,5,1,Barcelona,None,None,"Samitier, Samitier, Samitier, Samitier, Piera",Monjardín


In [318]:
new_df['League'] = 'Copa Del Rey'
new_df.to_csv("Datasets/Cleaned_ds/copa_del_rey.csv", index=False)

# **COPA DE LA LIGA**


In [73]:
file = pd.read_csv("Datasets/el_clasico_matches_copa_de_la_liga.csv")
df = pd.DataFrame(file)

In [74]:
df.head()

,Season,Round,Round.1,Home team,Away team,Score (FT/HT),Goals (home),Goals (away)
0,1982–83,Final,First leg,Real Madrid,Barcelona,2–2 (0–0),"Del Bosque (62), Juanito (68 p.)","Carrasco (53), Maradona (57)"
1,1982–83,Final,Second leg,Barcelona,Real Madrid,2–1 (2–0),"Maradona (19 p.), Alexanko (25)",Santillana (80)
2,1984–85,Quarter-finals,First leg,Barcelona,Real Madrid,2–2 (2–0),"Clos (40), Marcos (44)","Valdano (67), Juanito (75)"
3,1984–85,Quarter-finals,Second leg,Real Madrid,Barcelona,1–1 (0–0) (4–1 p.),Valdano (83),Moratalla (57)
4,1985–86,Second round,First leg,Barcelona,Real Madrid,2–2 (1–1),"Clos (24), Archibald (50)","Pardeza (36), Cholo (52)"


In [75]:
df = scorers(df)
df = transform_match_data(df)
df = goals(df)
df["Winner"] = df.apply(determine_winner, axis=1)
df["Venue"] = df["Home team"].map(venue_map)

In [76]:
columns={
            "Season": "Date",
            "Barcelona Goals": "Barcelona_Goals", 
            "Real Madrid Goals": "Real_Madrid_Goals",
            "Barcelona Scorers Clean" : "Barcelona_Scorers",
            "Real Madrid Scorers Clean": "Real_Madrid_Scorers"
         }
new_df = df.filter(items=["Season", "Venue", "Barcelona Goals", "Real Madrid Goals", "Winner", "Real_Madrid_Own_Goals", "Barcelona_Own_Goals","Barcelona Scorers Clean", "Real Madrid Scorers Clean"]).rename(columns = columns)

In [77]:
new_df['League'] = 'Copa De La Liga'
new_df.to_csv("Datasets/Cleaned_ds/copa_de_la_liga.csv", index=False)

# **SUPERCOPA**


In [78]:
file = pd.read_csv("Datasets/el_clasico_matches_supercopa.csv")
df = pd.DataFrame(file)

In [79]:
df.head()

,Season,Round,Home team,Away team,Score (FT/HT),Goals (home),Goals (away)
0,1988–89,First leg,Real Madrid,Barcelona,2–0 (0–0),"Míchel (51), Hugo Sánchez (78)",NaN
1,1988–89,Second leg,Barcelona,Real Madrid,2–1 (1–1),"Bakero (37, 78)",Butragueño (15)
2,1990–91,First leg,Barcelona,Real Madrid,0–1 (0–0),NaN,Míchel (55)
3,1990–91,Second leg,Real Madrid,Barcelona,4–1 (2–1),"Butragueño (21, 44), Hugo Sánchez (56), Aragón...",Goikoetxea (20)
4,1993–94,First leg,Real Madrid,Barcelona,3–1 (1–1),"Alfonso (33, 85), Zamorano (55)",Stoichkov (15)


In [80]:
df = scorers(df)
df = transform_match_data(df)
df = goals(df)
df["Winner"] = df.apply(determine_winner, axis=1)
df["Venue"] = df["Home team"].map(venue_map)

In [81]:
columns={
            "Season": "Year",
            "Barcelona Goals": "Barcelona_Goals", 
            "Real Madrid Goals": "Real_Madrid_Goals",
            "Barcelona Scorers Clean" : "Barcelona_Scorers",
            "Real Madrid Scorers Clean": "Real_Madrid_Scorers"
         }
new_df = df.filter(items=["Season", "Venue", "Barcelona Goals", "Real Madrid Goals", "Winner", "Real_Madrid_Own_Goals", "Barcelona_Own_Goals","Barcelona Scorers Clean", "Real Madrid Scorers Clean"]).rename(columns = columns)

In [82]:
new_df['League'] = 'Supercopa'
new_df.to_csv("Datasets/Cleaned_ds/supercopa.csv", index=False)

# **Dates Dataset**


In [24]:
file = pd.read_csv("Datasets/Dates_df.csv", encoding="ISO-8859-1")
df = pd.DataFrame(file)

In [25]:
df = df.dropna(subset=["Result"])

In [26]:
df.head()

,Date,Home team,Away team,Result
0,"Sat, Jan 11, 2025",Real Madrid,Barcelona,02:05
1,"Fri, Oct 25, 2024",Real Madrid (2.),Barcelona (1.),00:04
2,"Sat, Apr 20, 2024",Real Madrid (1.),Barcelona (2.),03:02
3,"Sat, Jan 13, 2024",Real Madrid,Barcelona,04:01
4,"Fri, Oct 27, 2023",Barcelona (4.),Real Madrid (2.),01:02


In [27]:
df['Date'] = pd.to_datetime(df['Date'], format='%a, %b %d, %Y').dt.strftime('%d-%m-%Y')


In [28]:
df['Year'] = pd.to_datetime(df['Date'], format='%d-%m-%Y').dt.year

In [29]:
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
df["Date"] = df["Date"] + pd.Timedelta(days=1)
df["Date"] = df["Date"].dt.strftime("%d-%m-%Y")


In [30]:
df = pd.DataFrame(df)

# Remove bracketed values from Home team and Away team
df['Home team'] = df['Home team'].apply(lambda x: re.sub(r'\s*\(.*?\)', '', x))
df['Away team'] = df['Away team'].apply(lambda x: re.sub(r'\s*\(.*?\)', '', x))

# Convert Result format from 02:05 to 2-5
df[['Home Score', 'Away Score']] = df['Result'].str.split(':', expand=True)
df['Home Score'] = df['Home Score'].astype(int)
df['Away Score'] = df['Away Score'].astype(int)
df['Result'] = df['Home Score'].astype(str) + '-' + df['Away Score'].astype(str)

# Determine the winner
def get_winner(row):
    if row['Home Score'] > row['Away Score']:
        return row['Home team']
    elif row['Home Score'] < row['Away Score']:
        return row['Away team']
    else:
        return 'Draw'

df['Winner'] = df.apply(get_winner, axis=1)




In [31]:
df.head()

,Date,Home team,Away team,Result,Year,Home Score,Away Score,Winner
0,12-01-2025,Real Madrid,Barcelona,2-5,2025,2,5,Barcelona
1,26-10-2024,Real Madrid,Barcelona,0-4,2024,0,4,Barcelona
2,21-04-2024,Real Madrid,Barcelona,3-2,2024,3,2,Real Madrid
3,14-01-2024,Real Madrid,Barcelona,4-1,2024,4,1,Real Madrid
4,28-10-2023,Barcelona,Real Madrid,1-2,2023,1,2,Real Madrid


In [32]:
df["Barcelona Goals"] = df.apply(lambda row: row["Home Score"] if row["Home team"] == "Barcelona" else row["Away Score"], axis=1)
df["Real Madrid Goals"] = df.apply(lambda row: row["Home Score"] if row["Home team"] == "Real Madrid" else row["Away Score"], axis=1)


In [33]:
df.head()

,Date,Home team,Away team,Result,Year,Home Score,Away Score,Winner,Barcelona Goals,Real Madrid Goals
0,12-01-2025,Real Madrid,Barcelona,2-5,2025,2,5,Barcelona,5,2
1,26-10-2024,Real Madrid,Barcelona,0-4,2024,0,4,Barcelona,4,0
2,21-04-2024,Real Madrid,Barcelona,3-2,2024,3,2,Real Madrid,2,3
3,14-01-2024,Real Madrid,Barcelona,4-1,2024,4,1,Real Madrid,1,4
4,28-10-2023,Barcelona,Real Madrid,1-2,2023,1,2,Real Madrid,1,2


In [34]:
df["Winner"].unique()

array(['Barcelona', 'Real Madrid', 'Draw', 'FC Barcelona', 'Madrid FC'],
      dtype=object)

In [35]:
df["Winner"] = df["Winner"].replace("FC Barcelona", "Barcelona")
df["Winner"] = df["Winner"].replace("Madrid FC", "Real Madrid")

In [36]:
columns={
            "Barcelona Goals": "Barcelona_Goals", 
            "Real Madrid Goals": "Real_Madrid_Goals",
         }
new_df = df.filter(items=["Date","Year", "Barcelona Goals", "Real Madrid Goals", "Winner"]).rename(columns = columns)

In [37]:
new_df.head()

,Date,Year,Barcelona_Goals,Real_Madrid_Goals,Winner
0,12-01-2025,2025,5,2,Barcelona
1,26-10-2024,2024,4,0,Barcelona
2,21-04-2024,2024,2,3,Real Madrid
3,14-01-2024,2024,1,4,Real Madrid
4,28-10-2023,2023,1,2,Real Madrid


In [38]:
new_df['Date'] = pd.to_datetime(new_df['Date'], format='%d-%m-%Y')
# Sort DataFrame by Date (Ascending)
df_sorted = new_df.sort_values(by='Date', ascending=True).reset_index(drop=True)

In [39]:
df_sorted.head()

,Date,Year,Barcelona_Goals,Real_Madrid_Goals,Winner
0,1902-05-13,1902,1,1,Barcelona
1,1916-03-26,1916,1,1,Barcelona
2,1916-04-02,1916,1,1,Real Madrid
3,1916-04-13,1916,6,6,Draw
4,1916-04-15,1916,2,2,Real Madrid


In [41]:
df_sorted.to_csv("Datasets/dates_df_cleaned.csv", index=False)

# **ALL GAMES**


In [78]:
import pandas as pd
import os

def combine_csv_files(directory_path, output_file):
    dfs = []
    for csv_file in os.listdir(directory_path):
        if csv_file.endswith(".csv"):
            file_path = os.path.join(directory_path, csv_file)
            try:
                df = pd.read_csv(file_path)  # Change encoding
                df['source_file'] = csv_file.replace('.csv', '')  # Add source column
                dfs.append(df)
            except UnicodeDecodeError:
                print(f"Encoding error in file: {csv_file}. Skipping it.")

    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df.to_csv(output_file, index=False)

# Call function
directory = "Datasets/Cleaned_ds"
output_file = "Datasets/combined_soccer_data.csv"
combine_csv_files(directory, output_file)


# **Add Dates**


In [332]:
file1 = pd.read_csv("Datasets/dates_df_cleaned.csv")
file2 = pd.read_csv("Datasets/combined_soccer_data.csv")
df1 = pd.DataFrame(file1)
df2 = pd.DataFrame(file2)

In [333]:
df1.head(4)

,Date,Year,Barcelona_Goals,Real_Madrid_Goals,Winner
0,12-01-2025,2025,5,2,Barcelona
1,26-10-2024,2024,4,0,Barcelona
2,21-04-2024,2024,2,3,Real Madrid
3,14-01-2024,2024,1,4,Real Madrid


In [334]:
df2.head(4)

,Year,Venue,Barcelona_Goals,Real_Madrid_Goals,Winner,Real_Madrid_Own_Goals,Barcelona_Own_Goals,Barcelona_Scorers,Real_Madrid_Scorers,League,source_file,Date
0,1916,Camp Nou,2,1,Barcelona,NaN,NaN,"Alcántara, Martínez",Petit,Copa Del Rey,copa_del_rey,NaN
1,1916,Santiago Bernabéu Stadium,1,4,Real Madrid,NaN,NaN,Martínez,Petit,Copa Del Rey,copa_del_rey,NaN
2,1916,Santiago Bernabéu Stadium,6,6,Draw,NaN,NaN,"Alcántara, Alcántara, Alcántara, Mallorquí, Ba...","Belaunde, Belaunde, Belaunde",Copa Del Rey,copa_del_rey,NaN
3,1916,Santiago Bernabéu Stadium,2,4,Real Madrid,NaN,NaN,"Martínez, Martínez","Bernabéu, Zabalo, Aranguren, Aranguren",Copa Del Rey,copa_del_rey,NaN


In [335]:
df1.shape

(259, 5)

In [337]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Date               259 non-null    object
 1   Year               259 non-null    int64 
 2   Barcelona_Goals    259 non-null    int64 
 3   Real_Madrid_Goals  259 non-null    int64 
 4   Winner             259 non-null    object
dtypes: int64(3), object(2)
memory usage: 10.2+ KB


In [338]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Year                   259 non-null    int64 
 1   Venue                  254 non-null    object
 2   Barcelona_Goals        259 non-null    int64 
 3   Real_Madrid_Goals      259 non-null    int64 
 4   Winner                 259 non-null    object
 5   Real_Madrid_Own_Goals  8 non-null      object
 6   Barcelona_Own_Goals    5 non-null      object
 7   Barcelona_Scorers      195 non-null    object
 8   Real_Madrid_Scorers    188 non-null    object
 9   League                 259 non-null    object
 10  source_file            259 non-null    object
 11  Date                   190 non-null    object
dtypes: int64(3), object(9)
memory usage: 24.4+ KB


In [336]:
df2.shape

(259, 12)

In [339]:
merged_df = df2.merge(df1, on=["Year", "Barcelona_Goals", "Real_Madrid_Goals", "Winner"], how="right")

In [342]:
merged_df = df2.merge(df1,
                      on=['Year', 'Barcelona_Goals', 'Real_Madrid_Goals'], 
                      how='left')

In [343]:
merged_df.shape

(284, 14)

In [340]:
merged_df.head()

,Year,Venue,Barcelona_Goals,Real_Madrid_Goals,Winner,Real_Madrid_Own_Goals,Barcelona_Own_Goals,Barcelona_Scorers,Real_Madrid_Scorers,League,source_file,Date_x,Date_y
0,2025,Santiago Bernabéu Stadium,5,2,Barcelona,NaN,NaN,"Yamal, Raphinha, Raphinha, Balde","Mbappé, Rodrygo",Supercopa,supercopa,NaN,12-01-2025
1,2024,Santiago Bernabéu Stadium,4,0,Barcelona,NaN,NaN,"Lewandowski, Lewandowski, Yamal, Raphinha",NaN,La Liga,la_liga,26-10-2024,26-10-2024
2,2024,Santiago Bernabéu Stadium,2,3,Real Madrid,NaN,NaN,"Christensen, López","Vázquez, Bellingham",La Liga,la_liga,21-04-2024,21-04-2024
3,2024,Santiago Bernabéu Stadium,1,4,Real Madrid,NaN,NaN,Lewandowski,Rodrygo,Supercopa,supercopa,NaN,14-01-2024
4,2023,Camp Nou,1,2,Real Madrid,NaN,NaN,Gündoğan,"Bellingham, Bellingham",La Liga,la_liga,28-10-2023,28-10-2023


In [347]:
merged_df.to_csv("Datasets/Cleaned_ds/merged_df.csv", index=False)

In [344]:
def merge_clasico_data(df1, df2):
    # Rename columns to ensure consistency
    df1 = df1.rename(columns={
        'Barcelona_Goals': 'Barcelona_Goals',
        'Real_Madrid_Goals': 'Real_Madrid_Goals'
    })
    
    # Create a match identifier using all common columns
    match_columns = ['Year', 'Barcelona_Goals', 'Real_Madrid_Goals', 'Winner']
    
    # Create match signatures for both dataframes
    df1['match_signature'] = df1[match_columns].astype(str).agg('-'.join, axis=1)
    df2['match_signature'] = df2[match_columns].astype(str).agg('-'.join, axis=1)
    
    # Create a mapping dictionary from match signature to date
    date_mapping = pd.Series(df1['Date'].values, index=df1['match_signature']).to_dict()
    
    # Add Date column to df2
    df2['Date'] = df2['match_signature'].map(date_mapping)
    
    # Drop the temporary match signature column
    df2 = df2.drop('match_signature', axis=1)
    
    # Sort the dataframe by date
    df2 = df2.sort_values('Date', ascending=False)
    
    return df2


merged_df = merge_clasico_data(df1, df2)

In [345]:
merged_df.shape

(259, 12)

In [346]:
merged_df.head()

,Year,Venue,Barcelona_Goals,Real_Madrid_Goals,Winner,Real_Madrid_Own_Goals,Barcelona_Own_Goals,Barcelona_Scorers,Real_Madrid_Scorers,League,source_file,Date
155,1987,Camp Nou,3,2,Barcelona,NaN,NaN,"Lineker, Lineker, Lineker",Valdano,La Liga,la_liga,31-01-1987
151,1984,Camp Nou,3,2,Barcelona,NaN,NaN,"Gerardo, Migueli, Vigo","Sanchís, Butragueño",La Liga,la_liga,30-12-1984
142,1980,Camp Nou,2,1,Barcelona,NaN,NaN,"Schuster, Quini",Juanito,La Liga,la_liga,30-11-1980
174,1995,Santiago Bernabéu Stadium,1,1,Draw,NaN,NaN,Roger,Raúl,La Liga,la_liga,30-09-1995
106,1962,Santiago Bernabéu Stadium,0,2,Real Madrid,NaN,NaN,NaN,"Stéfano, Stéfano",La Liga,la_liga,30-09-1962
